## **Preface**
This guide aims to learn about USD asset creation with python, and build familiarty with USD Python Library. 

## **Prerequisites**
Python examples in this guide requires only having `usd-core` library.

### **Install the `usd-core` Python package**

Install the [`usd-core`](https://pypi.org/project/usd-core/) Python package providing the core USD libraries. Note that it does not provide any of the optional plugins or imaging features from the complete USD distribution.

In [ ]:
! pip install usd-core

# See https://pypi.org/project/usd-core/#history for a list of supported USD
# versions.

## **USD Termnial commands**
I'm using `usdview` command to examine my generated usd files.
To avoid any headaches, I'm using `Houdini 19.5 Command Line Tools` which shipped with USD already.

Alternatively, you can use 
- [pre-built USD library](https://developer.nvidia.com/usd#libraries-and-tools)
- `usdview` shipped with Omniverse, [Get Omniverse](https://www.nvidia.com/en-us/omniverse/download/) then from `exchange` tab find `usdview`

## **Build Assets with python**

### **Warm-up**

Let's create a simple asset as a warm-up where everything is defined in one USD layer.
In this exercise:
```
{asset-name}.usda
  └── Xform {asset-name}
        ├── Variant Sets : [model, material]
        ├── Scope Geo
        |       └── Cube
        |            ├── add scale property
        |            └── append it to transform order property
        ├── Scope Mtl
        |        └── Red Material
        |              ├── connect sufrace output to the output surface of Red Shader 
        |              └── Red Shader
        |                    ├── set shader id
        |                    ├── set diffuse color
        |                    └── create output surface
        ├── model Variant Set
        |        └── one variant
        |                └── over Geo
        |                      └── over Cube
        |                            └── set scale property
        └── material Variant Set
                └── one variant
                        └── over Geo
                              └── over Cube
                                    ├── apply material binding api schema
                                    └── set material binding

```

In [19]:
from pxr import Usd, Sdf, UsdGeom, UsdShade

# Crteate Stage
stage = Usd.Stage.CreateInMemory()

# Create Main prim
mainPrim = UsdGeom.Xform.Define(stage, '/Cube_Asset')

# Create Geo Scope prim to group our geometry 
geoScope = UsdGeom.Scope.Define(stage, '/Cube_Asset/Geo')  

# Create a Cube prim inside Geo
cubePrim = UsdGeom.Cube.Define(stage, '/Cube_Asset/Geo/Cube')

# Add scale property and append it to xform order
scaleOp = cubePrim.AddScaleOp()  

# Create model Variant Set inside main prim
modelVariantSet = mainPrim.GetPrim().GetVariantSets().AddVariantSet('model')  

# Add new model variant
modelVariantSet.AddVariant('tall')  

# Set model variant selection
modelVariantSet.SetVariantSelection('tall')  

# Modifiy current model variant selection
with modelVariantSet.GetVariantEditContext():
    # Set Y scale to 2 to my cube. it will automatically create over prims
    scaleOp.Set((1, 2, 1))

# Create Mtl Scope
materialScope = UsdGeom.Scope.Define(stage, '/Cube_Asset/Mtl')

# Create a material prim inside Mtl
redMaterial = UsdShade.Material.Define(stage, '/Cube_Asset/Mtl/Red')

# Create a shader prim inside the material prim
redShaderPreview = UsdShade.Shader.Define(stage, '/Cube_Asset/Mtl/Red/RedShader')

# Connect material surface to the surface output of the shader 
redMaterial.CreateSurfaceOutput().ConnectToSource(redShaderPreview.ConnectableAPI(), "surface")

# Set Shader Id (material type)
redShaderPreview.CreateIdAttr("UsdPreviewSurface")

# Set diffuse color to (1,0,0) which means red.
redShaderPreview.CreateInput('diffuseColor', Sdf.ValueTypeNames.Color3f).Set((1, 0, 0))

# Create material Variant Set
materialVariantSet = mainPrim.GetPrim().GetVariantSets().AddVariantSet('material')

# Add new material variant
materialVariantSet.AddVariant('red')

# Set material variant selection
materialVariantSet.SetVariantSelection('red')

# Modifiy current material variant selection
with materialVariantSet.GetVariantEditContext():
    # Apply material variant to my cube. it will automatically create over prims
    cubePrim.GetPrim().ApplyAPI("MaterialBindingAPI")

    # Add material binding to my cube
    UsdShade.MaterialBindingAPI(cubePrim.GetPrim()).Bind(redMaterial)

# Print stage
print(stage.GetRootLayer().ExportToString())

# Export Stage to a usda file
stage.GetRootLayer().Export('examples/build_assets_with_python_1/warmup/cube_asset.usda')

#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string material = "red"
        string model = "tall"
    }
    prepend variantSets = ["model", "material"]
)
{
    def Scope "Geo"
    {
        def Cube "Cube"
        {
            float3 xformOp:scale
            uniform token[] xformOpOrder = ["xformOp:scale"]
        }
    }

    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/RedShader.outputs:surface>

            def Shader "RedShader"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }
    }
    variantSet "model" = {
        "tall" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 2, 1)
                }
            }

        }
    }
    variantSet "material" = {
        

True

**Result:**

![](../resources/build_assets_with_python_1/warmup.png)

### **Example 2 : variant Geo and variant Mtl layers as payload**
> This is exactly the same last example in [USD_Asset_Structure_1](USD_Asset_Structure_1.ipynb)

![](../resources/asset_structure/asset_structure.png)

#### **Geo Layer**

In [36]:
# Create Geom Layer
from pxr import Usd, UsdGeom

stage = Usd.Stage.CreateInMemory()

asset = UsdGeom.Xform.Define(stage, '/Cube_Asset')
geo_scope_prim = UsdGeom.Scope.Define(stage, asset.GetPath().AppendPath('Geo'))
cube_prim = UsdGeom.Cube.Define(stage, geo_scope_prim.GetPath().AppendPath('Cube'))

# colorAttr = cubePrim.GetDisplayColorAttr()
# colorAttr.Set([color])

scale_op = cube_prim.AddScaleOp()

# Add the VariantSet
model_variant_set = asset.GetPrim().GetVariantSets().AddVariantSet('model')

# Add variants to the VariantSet
model_variant_set.AddVariant('tall')
model_variant_set.AddVariant('short')
model_variant_set.AddVariant('normal')

# Set the variant values
model_variant_set.SetVariantSelection('normal')
with model_variant_set.GetVariantEditContext():
    # colorAttr.Set([(1,0,0)])
    scale_op.Set((1, 1, 1))

model_variant_set.SetVariantSelection('short')
with model_variant_set.GetVariantEditContext():
    # colorAttr.Set([(0,1,0)])
    scale_op.Set((1, 0.5, 1))

model_variant_set.SetVariantSelection('tall')
with model_variant_set.GetVariantEditContext():
    # colorAttr.Set([(0,0,1)])
    scale_op.Set((1, 2, 1))

# Set the model variant selection to normal
model_variant_set.SetVariantSelection('normal')


print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_1/asset_1_variant_geo_variant_mtl/geo.usda')

#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string model = "normal"
    }
    prepend variantSets = "model"
)
{
    def Scope "Geo"
    {
        def Cube "Cube"
        {
            float3 xformOp:scale
            uniform token[] xformOpOrder = ["xformOp:scale"]
        }
    }
    variantSet "model" = {
        "normal" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 1, 1)
                }
            }

        }
        "short" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 0.5, 1)
                }
            }

        }
        "tall" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 2, 1)
                }
            }

        }
    }
}




True

#### **Mtl Layer**


In [21]:
# Create Material Layer
# Material X won't be used in this example as it's not available inside usd-core.
from pxr import Usd, Sdf, UsdGeom, UsdShade

def create_material(parent_prim, mat_name, color_value): 
    material = UsdShade.Material.Define(stage, parent_prim.GetPath().AppendPath('{0}'.format(mat_name)) )
    shader_preview = UsdShade.Shader.Define(stage, material.GetPath().AppendPath('{0}_Shader'.format(mat_name)))
    material.CreateSurfaceOutput().ConnectToSource(shader_preview.ConnectableAPI(), "surface")
    shader_preview.CreateIdAttr("UsdPreviewSurface")
    shader_preview.CreateInput('diffuseColor',Sdf.ValueTypeNames.Color3f).Set(color_value)

    return material


stage = Usd.Stage.CreateInMemory()
asset = UsdGeom.Xform.Define(stage, '/Cube_Asset')
mtl_scope_prim = UsdGeom.Scope.Define(stage, asset.GetPath().AppendPath('Mtl'))

# Create materials
red_material   = create_material(mtl_scope_prim, "Red"  , (1, 0, 0))
green_material = create_material(mtl_scope_prim, "Green", (0, 1, 0))
blue_material  = create_material(mtl_scope_prim, "Blue" , (0, 0, 1))

# Material variants are different binding variants.
material_variant_set = asset.GetPrim().GetVariantSets().AddVariantSet('material')

variants = {
    'red'   : red_material,
    'green' : green_material,
    'blue'  : blue_material
}

for variant_name, variant_value in variants.items() :
    material_variant_set.AddVariant(variant_name)
    material_variant_set.SetVariantSelection(variant_name)
    with material_variant_set.GetVariantEditContext():
        # Create geo prim override inside the variant.
        cube_prim = stage.OverridePrim(asset.GetPath().AppendPath('Geo/Cube'))  # Same prim hierarchy as in geo.usda
        cube_prim.ApplyAPI("MaterialBindingAPI")
        UsdShade.MaterialBindingAPI(cube_prim).Bind(variant_value)  # add material binding to geo override

# Set the material variant selection to red
material_variant_set.SetVariantSelection('red')

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_1/asset_1_variant_geo_variant_mtl/mtl.usda')

#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string material = "red"
    }
    prepend variantSets = "material"
)
{
    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/Red_Shader.outputs:surface>

            def Shader "Red_Shader"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }

        def Material "Green"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Green/Green_Shader.outputs:surface>

            def Shader "Green_Shader"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (0, 1, 0)
                token outputs:surface
            }
        }

        def Material "Blue"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Blue/Blue_Shade

True

#### **Payload layer**


In [31]:
# Payload
from pxr import Usd, Sdf, UsdGeom, UsdShade
    
name = "payload"
stage = Usd.Stage.CreateInMemory()
asset = UsdGeom.Xform.Define(stage, '/Cube_Asset')

asset.GetPrim().GetReferences().AddReference('./geo.usda', '/Cube_Asset')
asset.GetPrim().GetReferences().AddReference('./mtl.usda', '/Cube_Asset')

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_1/asset_1_variant_geo_variant_mtl/payload.usda')

#usda 1.0

def Xform "Cube_Asset" (
    prepend references = [
        @./geo.usda@</Cube_Asset>,
        @./mtl.usda@</Cube_Asset>
    ]
)
{
}




True

#### **Asset layer**

In [35]:
# Create asset with payload and variant selections
from pxr import Usd, Sdf, UsdGeom


stage = Usd.Stage.CreateInMemory()
asset = UsdGeom.Xform.Define(stage, '/Cube_Asset')

# Add payload
asset.GetPrim().GetPayloads().AddPayload('./payload.usda', '/Cube_Asset')

variants = {
    'model' : ['normal', 'short', 'tall'],
    'material' : ['red', 'green', 'blue']
}

# Loft variants
for variant_set_name, variants_list in variants.items():
    variant_set = asset.GetPrim().GetVariantSets().AddVariantSet(variant_set_name)
    for variant in variants_list:
        variant_set.AddVariant(variant)
        
    variant_set.SetVariantSelection(variants_list[0])

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_1/asset_1_variant_geo_variant_mtl/cube_asset.usda')

#usda 1.0

def Xform "Cube_Asset" (
    prepend payload = @./payload.usda@</Cube_Asset>
    variants = {
        string material = "red"
        string model = "normal"
    }
    prepend variantSets = ["model", "material"]
)
{
    variantSet "model" = {
        "normal" {

        }
        "short" {

        }
        "tall" {

        }
    }
    variantSet "material" = {
        "blue" {

        }
        "green" {

        }
        "red" {

        }
    }
}




True

**Result**
It's the exact same result as example 4 in Asset structure 1.
![](../resources/build_assets_with_python_1/asset_1_variant_geo_variant_mtl.gif)